In [35]:
#%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive
import glob, sys, os
from scipy.ndimage.interpolation import shift
from scipy.misc import imread
import skimage
from skimage import io
import numpy as np
from skimage.util import random_noise, img_as_uint
from loading_and_piv_funcs_new import *
import tifffile as tiff
import os, sys
from loading_and_piv_funcs_new import *
import glob
from datetime import datetime
from scipy.ndimage.interpolation import shift
from mpl_toolkits.mplot3d import Axes3D

In [36]:
for corr_method in ['fft', 'sad']:
    
    for y in ['y000','y300','y600','y900','y1200']:
        
        img_path = ('/home/vytas/1phd/rawdata/bead_experiments/2017-12-15-7um-beads-zscans-static-correct-mirror/'
                    + y + '/  Retiga 1350B/')    
        imgs = sorted(glob.glob(img_path + "*.tif"))
        name = '2017-DEC-15th-7um-1024-512-crop-7um-' + y

        window_height,window_width = 128, 128
        window_big_height, window_big_width = 256,256
        # Small IW overlap
        overlap_height,overlap_width = 0, 0
        #Big IW overlap
        overlap_big_height = window_big_height - (window_height - overlap_height)
        overlap_big_width = window_big_width - (window_width - overlap_width)
        #Centering the IWs
        subwindow_inset_height = (window_big_height-window_height)/2
        subwindow_inset_width = (window_big_width - window_width)/2


        for frame_delta in range(1,5):


            #how many frames for 1 um z change
            z_step_multiplier = 10
            synthetic_px_shift = 24
            corr_method =  corr_method#'fft'#'sad' # 'fft'
            subpixel_method = 'gaussian'#'gaussian'#'parabolic'

            wkdir = ('./corr_avg/' + name + 'frame_delta-' + str(frame_delta) + '-'
            + '_' + corr_method + '_' + subpixel_method + '_'
            + str(window_height) + '_' + str(window_width) + '_HW_' + str(window_big_height)
            + '_' +str(window_big_width) +  '_ol_hw_' + str(overlap_height) + '_'
            + str(overlap_width) + '/')
            folders = ['xyuvms2n', 'corr', 'metadata']
            xyuv_dir,corr_avg_dir,meta_dir = [folder_writer(wkdir, folder) for folder in folders]
            ###PARAMS###
            data_type = 'int32'

            mode =  'valid' # 'full' # "choose 'full' for same sized windows"
            dt = 1.0
            px_size = 1.0 #0.65 # effective (i.e. (true pixel size in camera)/magnification) pixel size in microns
            #'gaussian'#'parabolic'
            sig2noise_method = 'peak2peak'
            threshold = 1.000
            width = 3 #half width of the search area/mask around the peak

            initial_frame = 0
            mean_intensity_thres = 1e5
            square = False


            path_to_images = img_path
            img_files = imgs#sorted( glob.glob( os.path.join( os.path.abspath(path_to_images), '*.tif' ) ) )#[1200:2875]

            par_vals = dict( (name,eval(name)) for name in ['window_height','window_width',
            'subwindow_inset_height','subwindow_inset_width','overlap_height',
            'overlap_width','window_big_height','window_big_width','overlap_big_height',
            'overlap_big_width','corr_method','mode','dt','px_size','subpixel_method',
            'sig2noise_method','threshold',
            'width','frame_delta','initial_frame', 'path_to_images', 'mean_intensity_thres'])

            with open(str(wkdir) + '/pars_test.csv', 'wb') as f:
                writer = csv.writer(f,delimiter='=')
                for row in par_vals.iteritems():
                    writer.writerow(row)

            ### Should move the below into a seperate file. Just need to work out
            ### global variable stuff.
            nr_planes = 1
            initial_plane = 0

            skip_data = None
            #shift(imread(images[26*i]), shift = (0,5), mode = 'reflect')
            massive_corr = np.zeros((129,129))
            massive_corr_single_iws = np.zeros((21,129,129))

            for counter, i in enumerate( range(len(img_files)-frame_delta*z_step_multiplier) ):

                frame_a = scipy.misc.imread(img_files[i])[0:1024,490:(490+512)].astype(data_type)
                #frame_b = scipy.misc.imread(img_files[i+frame_delta]).astype(data_type)

                frame_b = shift( scipy.misc.imread(img_files[i+frame_delta*z_step_multiplier])[0:1024,490:(490+512)],
                                shift = (0,synthetic_px_shift), mode = 'reflect').astype(data_type)

                writer=csv.writer(open(meta_dir + '/frame_pairs.csv', 'a'))
                writer.writerow([os.path.basename(img_files[i]), 
                                 os.path.basename(img_files[i+frame_delta*z_step_multiplier])] )

                image_size = frame_a.shape
                #size of the "cropped" image
                image_size_small = (image_size[0] - 2*subwindow_inset_height , image_size[1] - 2*subwindow_inset_width)
                #Small IW centres.subwindow_inset_width/height are added to compensate for different sizes in IWs.
                x,y = (get_coordinates( image_size_small, window_height, window_width, overlap_height,
                                       overlap_width)[0] + subwindow_inset_width,
                       get_coordinates( image_size_small, window_height, window_width,
                                       overlap_height,overlap_width )[1] + subwindow_inset_height)
                #Big IW centres
                q,p = (get_coordinates(image_size, window_big_height,window_big_width,
                                       overlap_big_height,overlap_big_width)[0],
                       get_coordinates(image_size, window_big_height,window_big_width,
                                       overlap_big_height,overlap_big_width)[1])
                #Check if the IW's have matching centres.
                assert(np.array_equal(x, q) == True)
                assert(np.array_equal(y, p) == True)
                windows_a = moving_sub_window_array(frame_a, window_big_height,window_big_width,
                                                    overlap_big_height,overlap_big_width,
                                                    subwindow_inset_height,subwindow_inset_width).astype(data_type)

                windows_b = moving_window_array( frame_b,window_big_height,window_big_width,
                                                overlap_big_height,overlap_big_width).astype(data_type)

                corr_shape = correlate_windows( windows_a[0], windows_b[0], corr_method = corr_method, mode = mode).shape
                corr = np.zeros((windows_a.shape[0],corr_shape[0],corr_shape[1]))
                corr_big_single_iws = np.zeros((windows_a.shape[0],corr_shape[0],corr_shape[1]))
                corr_big = np.zeros((1,corr_shape[0],corr_shape[1]))

                assert(windows_a.shape[0] == windows_b.shape[0])
                n_rows, n_cols = get_field_shape ( frame_a.shape , window_big_height,
                                                  window_big_width, overlap_big_height,overlap_big_width )
                u = np.zeros(n_rows*n_cols).astype(np.float64)
                v = np.zeros(n_rows*n_cols).astype(np.float64)
                sig2noise = np.zeros(n_rows*n_cols).astype(np.float64)
                intTotIwA = np.zeros(n_rows*n_cols).astype(np.float64)
                intTotIwB = np.zeros(n_rows*n_cols).astype(np.float64)

                for j in range(windows_a.shape[0]):
                    corr[j] = correlate_windows( windows_a[j], windows_b[j], corr_method = corr_method, mode = mode)
                    intTotIwA[j] = np.sum(windows_a[j])
                    intTotIwB[j] = np.sum(windows_b[j])
                    assert(corr.dtype == np.float64)

                corr_big_single_iws = np.add(corr_big_single_iws, corr)
                #print "corr big shape: ", corr_big.shape
                #print "counter: ", counter, "corr big sum: ", np.sum(corr_big_single_iws)
                corr_big = np.sum(corr, axis = 0)
                #print "corr big shape: ", corr_big.shape
                #print "counter: ", counter, "corr big sum: ", np.sum(corr_big)



                if counter%((10*frame_delta + 300))==0:
                    #print "modulo:", 10*frame_delta + 300
                    #print "adding ", counter, "nth frame pair corr to the massive corr."
                    massive_corr = np.add(massive_corr, corr_big)
                    massive_corr_single_iws = np.add(massive_corr_single_iws, corr_big_single_iws)
                    np.save( corr_avg_dir  + "/massive_corr_pair_" + str("%04d" %counter), massive_corr)
                    np.save( corr_avg_dir  + "/massive_corr_SINGLE_IWS_pair_" + str("%04d" %counter), massive_corr_single_iws)
    # ############################################################
                    corr_array = massive_corr
                    shape_0, shape_1 = 129,129
                    figure = plt.figure()
                    data = corr_array
                    ax = Axes3D(figure)
                    nx, ny = shape_0,shape_1
                    xx = range(nx)
                    yy = range(ny)  
                    xmax = np.argmax(np.max(data, axis=0))
                    ymax = np.argmax(np.max(data, axis=1))        
                    ax.set_xlabel('x axis. Peak column = ' +str(xmax))
                    ax.set_ylabel('y axis. Peak row = ' + str(ymax))
                    X, Y = np.meshgrid(xx, yy)
                    ax.plot_surface(X , Y , data , rstride = 4, cstride = 4,alpha=0.7)
                    plt.savefig(wkdir + 'corr_matrix_' + str(counter) + '_fd_' + str(frame_delta) + '.png')
                    plt.close()
                    figure = plt.figure(figsize = (12,12))
                    data = corr_array
                    #print "data shape", data.shape
                    nx, ny = shape_0,shape_1
                    xx = range(nx)
                    yy = range(ny)  
                    xmax = np.argmax(np.max(data, axis=0))
                    ymax = np.argmax(np.max(data, axis=1))        
                    plt.xlabel('x axis. Peak column = ' +str(xmax))
                    plt.ylabel('y axis. Peak row = ' + str(ymax))
                    X, Y = np.meshgrid(xx, yy)
                    plt.imshow(data, cmap='jet', interpolation='none')
                    plt.colorbar()
                    plt.contour(data,30,colors='k')
                    #plt.xlim(150,300)
                    #plt.ylim(100,250)
                    #print "xmax,ymax: ", xmax,ymax
                    plt.scatter(xmax,ymax)

                    plt.savefig(wkdir + 'corr_heatmap_' + str(counter) + '_fd_' + str(frame_delta) + '.png')
                    plt.close()

                #np.save( corr_avg_dir  + "/pair_" + str("%04d" %counter), corr)

                for l in range(corr.shape[0]):#, desc = 'finding subpixel vals for each iw'):
                    row, col = find_subpixel_peak_position( corr[l], subpixel_method=subpixel_method,
                                                           corr_method = corr_method)
                    u[l], v[l] = px_size*(col - corr_shape[1]/2), -px_size*(row - corr_shape[0]/2)
                    sig2noise[l] = sig2noise_ratio( corr[l], sig2noise_method=sig2noise_method,
                                                   corr_method= corr_method, width=width, square=square)

                u,v,sig2noise = u.reshape(n_rows, n_cols), v.reshape(n_rows, n_cols),sig2noise.reshape(n_rows, n_cols)
                u,v, mask = sig2noise_val( u, v, sig2noise, threshold)
                #print counter, u[0][0], synthetic_px_shift - u[0][0],

                #writer=csv.writer(open(meta_dir + '/du.csv', 'a'))
                #writer.writerow([synthetic_px_shift - u[0][0]] )

                xyuvms2n_single = xyuv_dir + "/xyuvms2n_pair_single_pairs_no_corr_av_" + str("%06d" %counter) + ".txt"
                #save( x, y, u, v, mask, sig2noise, xyuvms2n_single, fmt='%8.4f', delimiter='\t' )
                saveWithInt( x, y, u, v, mask, sig2noise,intTotIwA,intTotIwB, xyuvms2n_single, fmt='%8.4f', delimiter='\t')


                ###########corr big#########

                row, col = find_subpixel_peak_position( corr_big, subpixel_method=subpixel_method,
                                                           corr_method = corr_method)
                u_ca, v_ca = px_size*(col - corr_shape[1]/2), -px_size*(row - corr_shape[0]/2)

                #print "corr av result: ", counter, u, synthetic_px_shift - u

                writer=csv.writer(open(meta_dir + '/du_ca.csv', 'a'))
                writer.writerow([np.abs(synthetic_px_shift - u_ca)] )


            ########################massive corr single iws##############
            u = np.zeros(n_rows*n_cols).astype(np.float64)
            v = np.zeros(n_rows*n_cols).astype(np.float64)
            sig2noise = np.zeros(n_rows*n_cols).astype(np.float64)
            intTotIwA = np.zeros(n_rows*n_cols).astype(np.float64)
            intTotIwB = np.zeros(n_rows*n_cols).astype(np.float64)
            for l in range(massive_corr_single_iws.shape[0]):
                #print "massive_corr_single_iws.shape[0]", massive_corr_single_iws.shape[0]
                row, col = find_subpixel_peak_position( massive_corr_single_iws[l], subpixel_method=subpixel_method,
                                                       corr_method = corr_method)
                #print row,col
                u[l], v[l] = px_size*(col - corr_shape[1]/2), -px_size*(row - corr_shape[0]/2)
                sig2noise[l] = sig2noise_ratio( corr[l], sig2noise_method=sig2noise_method,
                                               corr_method= corr_method, width=width, square=square)

            u,v,sig2noise = u.reshape(n_rows, n_cols), v.reshape(n_rows, n_cols),sig2noise.reshape(n_rows, n_cols)
            u,v, mask = sig2noise_val( u, v, sig2noise, threshold)
            xyuvms2n_single_iw_ca = xyuv_dir + "/single_iw_CAV_xyuvms2n" + ".txt"
            saveWithInt( x, y, u, v, mask, sig2noise,intTotIwA,intTotIwB, xyuvms2n_single_iw_ca, fmt='%8.4f', delimiter='\t')
            writer=csv.writer(open(meta_dir + '/du_massive_corr_single_iw_frame_delta_' + str(frame_delta) + '.csv', 'a'))
            for row in u:
                for val in row:
                    writer.writerow([val, np.abs(synthetic_px_shift - val)])



            ###########################massive corr#####################
            row, col = find_subpixel_peak_position( massive_corr, subpixel_method=subpixel_method,
                                                       corr_method = corr_method)

            u_ca_mass, v_ca_mass = px_size*(col - corr_shape[1]/2), -px_size*(row - corr_shape[0]/2)
            #print u_ca_mass
            writer=csv.writer(open(meta_dir + '/du_massive_corr.csv', 'a'))
            writer.writerow([synthetic_px_shift - u_ca_mass,np.abs(synthetic_px_shift - u_ca_mass)] )

            print frame_delta, "massive corr result: ", counter, u_ca_mass, synthetic_px_shift - u_ca_mass
            np.save( corr_avg_dir  + "/frame_delta_massive_corr_" + str("%03d" %frame_delta), massive_corr)

1 massive corr result:  2990 24.1497502614 -0.149750261411
2 massive corr result:  2980 24.1438653828 -0.14386538282
3 massive corr result:  2970 24.1286421262 -0.12864212616
4 massive corr result:  2960 24.046593318 -0.0465933179619
1 massive corr result:  2991 24.1681027785 -0.168102778508
2 massive corr result:  2981 24.1585436564 -0.158543656442
3 massive corr result:  2971 24.2234798504 -0.223479850379
4 massive corr result:  2961 24.2573250137 -0.257325013749
1 massive corr result:  2991 24.0762742573 -0.0762742573435
2 massive corr result:  2981 24.1789388658 -0.178938865769
3 massive corr result:  2971 24.3236696979 -0.323669697877
4 massive corr result:  2961 24.1395454715 -0.13954547152
1 massive corr result:  2991 23.825041305 0.174958694969
2 massive corr result:  2981 23.9631116867 0.0368883133279
3 massive corr result:  2971 24.1565834821 -0.156583482086
4 massive corr result:  2961 24.3219947312 -0.321994731249
1 massive corr result:  2990 24.0769828372 -0.0769828372
2 m

In [ ]:
#1 massive corr result:  2989 24.0355591278 -0.0355591277624

#corrs = sorted(glob.glob( os.path.join( os.path.abspath(corr_avg_dir), '*.npy' ) ))
#corr_array = np.load(corrs[0])
corr_array = massive_corr
#shape_0,shape_1 =corr_array.shape[2], corr_array.shape[1]
shape_0,shape_1 =corr_array.shape[1], corr_array.shape[0]
#for single_corr in corrs[1:]:
 #   corr_array = np.add(corr_array, np.load(single_corr))
    #print("{:.2e}".format(np.amin(corr_array)),("{:.2e}".format(np.amax(corr_array)))) 
#massive_corr = np.load('/home/vytas/1repo_clones/padded_piv/corr_avg/dec-12-up-to-20um/piv_results/Dec12th-11-15-01-7um-beads-CROP-512-1_fft_gaussian_128_128_HW_256_256_ol_hw_0_0/corr/frame_delta_massive_corr_001.npy')

In [ ]:
#print u[0][0],v[0][0]
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
corr_array = massive_corr
shape_0, shape_1 = 129,129
figure = plt.figure()
data = corr_array
ax = Axes3D(figure)
nx, ny = shape_0,shape_1
xx = range(nx)
yy = range(ny)  
xmax = np.argmax(np.max(data, axis=0))
ymax = np.argmax(np.max(data, axis=1))        
ax.set_xlabel('x axis. Peak column = ' +str(xmax))
ax.set_ylabel('y axis. Peak row = ' + str(ymax))
X, Y = np.meshgrid(xx, yy)
ax.plot_surface(X , Y , data , rstride = 4, cstride = 4,alpha=0.7)
plt.show()

In [ ]:
figure = plt.figure(figsize = (24,24))
data = corr_array
print "data shape", data.shape
nx, ny = shape_0,shape_1
xx = range(nx)
yy = range(ny)  
xmax = np.argmax(np.max(data, axis=0))
ymax = np.argmax(np.max(data, axis=1))        
plt.xlabel('x axis. Peak column = ' +str(xmax))
plt.ylabel('y axis. Peak row = ' + str(ymax))
X, Y = np.meshgrid(xx, yy)
plt.imshow(data, cmap='jet', interpolation='none')
plt.colorbar()
plt.contour(data,30,colors='k')
#plt.xlim(150,300)
#plt.ylim(100,250)

plt.scatter(xmax,ymax)

plt.show()